In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import pickle

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

path=r"C:\Users\berid\python\yts mx project"

In [2]:
def return_movie_dict(movie_url):
    
    html=requests.get(movie_url,headers=headers).content
    soup=BeautifulSoup(html)

    try:
        title=soup.select_one('div[id="movie-info"] h1').text
    except:
        title=None

    try:
        poster=soup.select_one('div[id="movie-poster"] img')['src']
    except:
        poster=None

    try:
        year=soup.select('div[id="movie-info"] h2')[0].text
    except:
        year=None

    try:
        genres=soup.select('div[id="movie-info"] h2')[1].text
    except:
        genres=None

    try:
        imdb=soup.select('div[class="bottom-info"] div[class="rating-row"]')[-1].text
    except:
        imdb=None

    try:
        imdb_link=soup.select('div[class="bottom-info"] div[class="rating-row"]')[-1].a['href']
    except:
        imdb_link=None

    try:
        director=soup.select_one('div[class="directors"] span[itemprop="name"]').text
    except:
        director=None

    try:
        cast=[i.text for i in soup.select('div[class="actors"] div[class="list-cast"] a[class="name-cast"]')]
    except:
        cast=None

    try:
        plot=soup.select_one('div[id="synopsis"]').text
    except:
        plot=None

    try:
        tags=soup.select_one('div[class="row"] span[style="font-size:85%; color: #BCB8BC;"]').text
    except:
        tags=None

    movie_dict={'Movie URL':movie_url,'Title':title,'Poster':poster,'Year':year,'Genres':genres,'IMDB':imdb,'IMDB Link':imdb_link,'Director':director,'Cast':cast,'Plot':plot,'Tags':tags}

    return movie_dict

In [3]:
all_movie_urls=pickle.load(open(os.path.join(path,'scraped_data','all_yts_urls.pickle'),'rb'))

all_movie_urls=list(set(all_movie_urls))

print(f'URLs Total: {len(all_movie_urls)}')

URLs Total: 63545


In [4]:
import sqlite3

conn=sqlite3.connect(os.path.join(path,'scraped_data','yts.db'))
cursor=conn.cursor()

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS movie_details (
    MOVIE_URL TEXT,
    TITLE TEXT,
    POSTER TEXT,
    YEAR TEXT,
    GENRES TEXT,
    IMDB TEXT,
    IMDB_LINK TEXT,
    DIRECTOR TEXT,
    CAST TEXT,
    PLOT TEXT,
    TAGS TEXT
)
""")

conn.commit()

In [6]:
cursor.execute("SELECT DISTINCT MOVIE_URL FROM movie_details")
scraped_urls = [row[0] for row in cursor.fetchall()]

print(f'URLs scraped: {len(scraped_urls)}')

URLs scraped: 51475


In [7]:
urls_to_scrape=[url for url in all_movie_urls if url not in scraped_urls]
print(f'URLs to scrape: {len(urls_to_scrape)}')

URLs to scrape: 12070


In [8]:
for i,movie_url in enumerate(urls_to_scrape,start=1):
    
    try:
        movie_dict=return_movie_dict(movie_url)
    except:
        continue

    values=[str(v) for k,v in movie_dict.items()]

    cursor.execute('INSERT INTO movie_details (MOVIE_URL, TITLE, POSTER, YEAR, GENRES, IMDB, IMDB_LINK, DIRECTOR, CAST, PLOT, TAGS) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', values)
    conn.commit()

    print(f'Progress: {i}/{len(urls_to_scrape)}',end='\r')
